## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




In [59]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [38]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from pathlib import Path
import seaborn as sns

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

# plt.figure.max_open_warning = False

## Loading results

In [2]:
dir_path = Path('results/')
dz = 0.001

In [3]:
dt = 1e-8
mr = 1
name = 'neutralization.h5'
lenght = 0.011 # m
width = 0.005
cell_size = 0.001
temperature = 23e3 # K
nb_cells = 33
volume = cell_size*cell_size*dz*nb_cells


In [4]:
store = pd.HDFStore(dir_path/name)

In [5]:
store.keys()

['/collisions_with_walls',
 '/df',
 '/df_collision_background_gas',
 '/df_collision_with_walls',
 '/df_out_particles',
 '/mean_proba_gas',
 '/mean_proba_walls',
 '/total_deleted']

In [6]:
# defining each variable
df = store['df']
df_out_particles = store['df_out_particles']
df_collision_background_gas = store['df_collision_background_gas']
df_collision_with_walls = store['df_collision_with_walls']
total_deleted = store['total_deleted']
collisions_with_walls = store['collisions_with_walls']
mean_proba_walls = store['mean_proba_walls']
mean_proba_gas = store['mean_proba_gas']

In [7]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.


mass = physics.get_mass_part(53+1, 53, 74) # I
molecular_mass = 0.1269 # kg/mol

verbose=False

In [8]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature 23000.0 K : 1584.7563179678798 m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [9]:

res_x_0, res_y_0 = 11, 5

total_nb_cells = res_x_0*res_y_0

In [10]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df.index.unique().values.shape[0]
analysis.hist2d(ax, df, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Choosing frames to plot

In [11]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.0*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Proba evolution

In [12]:
fig, ax = plt.subplots(2)
# ax.set_yscale('log')
ax[0].plot(mean_proba_gas.index*dt, mean_proba_gas.values, label = 'gas')
ax[1].plot(mean_proba_walls.index*dt, mean_proba_walls.values, label = 'walls');
ax[0].legend();
ax[1].legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particles analysis

In [13]:
df_out_particles.head()

,x,y,vx,vy,vz,type
103,0.016047,0.002883,21751.642408,50.972980,585.330129,0.0
105,0.016156,0.002245,21650.361346,1402.021962,566.482263,0.0
105,0.016179,0.002119,21708.616919,613.180045,63.655224,0.0
105,0.016040,0.002337,21609.240420,-330.275600,-1270.573402,1.0
105,0.016116,0.002135,21644.289070,1625.036355,1911.020490,1.0


In [108]:
def convert_to_species(row):
    if(row['type'] == 0):
        return 'I-'
    else :
        return 'I'

In [109]:
df_out_particles['species'] = df_out_particles.apply(convert_to_species, axis = 1)

In [110]:
df_out_particles['v'] = np.sqrt(df_out_particles['vx']**2+df_out_particles['vy']**2+df_out_particles['vz']**2)
df_out_particles['energy'] = df_out_particles['v']**2
df_out_particles['angle'] = np.arctan(df_out_particles['vy']/df_out_particles['vx'])

In [111]:
ions_out = df_out_particles.loc[df_out_particles['type'] == 0]
neutral_out =  df_out_particles.loc[df_out_particles['type'] == 1]

In [105]:
results = df_out_particles[['type', 'x']].groupby('type').count();
print('Out particles neutralization percentage : {:.3} %'.format(100*results.iloc[1,0]/(results.iloc[1,0]+results.iloc[0,0])))

Out particles neutralization percentage : 80.1 %


In [112]:
vel_results = df_out_particles[['species', 'vx', 'vy', 'vz', 'v','energy']].groupby('species').agg('mean');
vel_results

,vx,vy,vz,v,energy
species,,,,,
I,17353.747223,-75.131617,-31.956131,18213.824344,3.675043e+08
I-,21144.442520,55.042993,17.692484,21392.902857,4.576711e+08


In [113]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.histplot(
    df_out_particles,
    x="vx", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);
# ax.set_xticks([2000, 5000, 10000, 20000]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.histplot(
    df_out_particles,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## df_collision_background_gas

In [116]:
df_collision_background_gas['species'] = df_collision_background_gas.apply(convert_to_species, axis = 1)

In [117]:
print('Number of collisions with background gas : {}'.format(df_collision_background_gas.shape[0]))
df_collision_background_gas.head()

Number of collisions with background gas : 4663


,x,y,type,species
1,0.000054,0.003384,0.0,I-
1,0.000029,0.004883,0.0,I-
1,0.000014,0.004905,0.0,I-
2,0.000073,0.002246,0.0,I-
3,0.000075,0.001645,0.0,I-


In [118]:
# Ion plot
sns.set_theme(style="ticks")
sns.jointplot(data = df_collision_background_gas.loc[df_collision_background_gas['type']==0], x='x', y='y', kind="hex", color="#4CB391");

# Neutral
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_background_gas.loc[df_collision_background_gas['type']==1], x='x', y='y', kind="hex", color="#4CB391");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
# both
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_background_gas, x='x', y='y', hue="type", s = 10) # , kind="hist") #, kind="hist")

## df_collision_with_walls

In [67]:
print('Number of collisions with walls : {}'.format(df_collision_with_walls.shape[0]))
df_collision_with_walls.head()

Number of collisions with walls : 20507


,x,y,type
1,0.000044,0.004996,0.0
2,0.000033,0.000002,0.0
3,0.000025,0.000002,0.0
4,0.000040,0.004998,0.0
5,0.000047,0.000005,0.0


In [70]:
ions = df_collision_with_walls.loc[df_collision_with_walls['type']==0]
fig, ax = plt.subplots()
ax.axis('equal')
scatter = ax.scatter(ions['x'], ions['y'], s = 0.005, c = ions['type']);
# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Species")
ax.add_artist(legend1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
sns.set_theme(style="ticks")
sns.jointplot(data = ions, x='x', y='y', kind="hex", color="#4CB391");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
# Neutral plot
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_with_walls, x='x', y='y', hue="type", s = 10) # , kind="hist") #, kind="hist")

## Particles analysis
### Plotting number of particles evolution

In [32]:
nb_parts = df['x'].groupby(df.index).agg('count').values

In [33]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts)
analysis.set_axis(ax, x = 'time', y = 'quantity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
sns.set_theme(style="ticks")
sns.jointplot(data = df, x='x', y='y', kind="hex", color="#4CB391");


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …